<a href="https://colab.research.google.com/github/VladimirApter/ml4se_course_2025/blob/main/homeworks/ml4se_course_hw6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Модель: ChatGPT 5**

**Промпт:**
Мне нужно провести исследование подходит ли CodeBERT для задачи clone detection. Я хочу проверить это следующим образом: взять примеры из датасета BigCloneBench (https://github.com/clonebench/BigCloneBench), для каждой пары фрагментов кода получить эмбеддинги фрагментов кода с помощью CodeBERT, а затем сравнить их, с помощью косинусного сходства. Если эмбеддинги окажутся близки по этой метрике, будем считать что ответ модели CodeBERT положительный, то есть данные фрагменты кода являются клонами. Сравнивая ответ модели CodeBERT с правильным ответом из датасета определим в каком проценте случаев модель CodeBERT дала правильный ответ и на основании этого вынесем вердикт по вопросу подходит ли CodeBERT для задачи clone detection. Напиши код на python в котором с помощью CodeBERT с Hugging face будет проведено это исследование

## Допущения эксперимента по Clone Detection с CodeBERT

При проведении эксперимента с использованием CodeBERT для задачи clone detection были сделаны следующие допущения:

1. **Выбор подвыборки датасета**
   - Для ускорения эксперимента и уменьшения потребления памяти мы использовали **только 1000 случайных пар фрагментов кода** из датасета BigCloneBench, вместо всего датасета (сотни тысяч пар).

2. **Метрика для оценки сходства**
   - В качестве меры близости между двумя фрагментами кода использовалась **косинусная схожесть эмбеддингов**, полученных от CodeBERT.
   - Пара считалась клоном, если косинусное сходство было **≥ 0.8** (порог выбран эмпирически).

3. **Эмбеддинги фрагментов кода**
   - Эмбеддинги получались через **mean pooling последнего слоя CodeBERT** по всем токенам.
   - Не использовалась дополнительная агрегация или архитектуры для парного сравнения.

4. **Ограничение по длине кода**
   - Для токенизации использовалась **усечённая длина до 256 токенов**, чтобы избежать переполнения памяти.

5. **Случайная выборка**
   - Подвыборка из 1000 пар была **случайной**, что может влиять на точность оценки; результаты могут немного отличаться при другой выборке.

---

Эти допущения упрощают эксперимент, позволяют избежать переполнения памяти и большой длительности по времени.


In [4]:
import torch
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
from tqdm import tqdm

MODEL_NAME = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

dataset = load_dataset("code_x_glue_cc_clone_detection_big_clone_bench", split="validation")
dataset = dataset.shuffle(seed=42).select(range(1000))


def get_embedding(code: str):
    tokens = tokenizer(code, return_tensors="pt", truncation=True, max_length=256)
    tokens = {k: v.to(device) for k, v in tokens.items()}
    with torch.no_grad():
        outputs = model(**tokens)
        emb = outputs.last_hidden_state.mean(dim=1)
    return emb.cpu().numpy()

def evaluate_model(dataset, sample_size=1000, threshold=0.8):
    n = min(sample_size, len(dataset))
    y_true = []
    y_pred = []

    for i in tqdm(range(n)):
        code1, code2, label = dataset[i]["func1"], dataset[i]["func2"], dataset[i]["label"]
        y_true.append(label)

        emb1 = get_embedding(code1)
        emb2 = get_embedding(code2)

        sim = cosine_similarity(emb1, emb2)[0][0]
        y_pred.append(1 if sim >= threshold else 0)

    acc = accuracy_score(y_true, y_pred)
    print(f"Accuracy: {acc:.4f}")
    return acc

accuracy = evaluate_model(dataset, sample_size=1000, threshold=0.8)
print(f"CodeBERT accuracy on {1000} BigCloneBench pairs: {accuracy:.4f}")


100%|██████████| 1000/1000 [09:53<00:00,  1.69it/s]

Accuracy: 0.1320
CodeBERT accuracy on 1000 BigCloneBench pairs: 0.1320


### Вывод по эксперименту с CodeBERT для Clone Detection

В эксперименте была использована предобученная модель **CodeBERT**, которая генерирует эмбеддинги фрагментов кода. Для 1000 случайных пар из датасета **BigCloneBench** была вычислена точность: косинусное сходство эмбеддингов ≥ 0.8 считалось признаком клонов.

**Результат эксперимента:**
- Accuracy: **0.1320** (13.2%)

**Вывод:**
- Предобученная CodeBERT без дообучения **не способна эффективно различать клоны и неклоны**.
